In [1]:
import base64
import os
import ipywidgets as wg
from IPython.display import display

In [2]:
user = wg.Text (description = 'WEkEO username')
password = wg.Password(placeholder='Enter password',description='WEkEO Password:')
display (user, password)

Text(value='', description='WEkEO username')

Password(description='WEkEO Password:', placeholder='Enter password')

In [25]:
url = 'https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker'
user = user.value
password = password.value

In [ ]:
parameters = 'url: '+url+'\nuser: '+user+'\npassword: '+password

In [ ]:
with open(os.path.join(os.environ['HOME'],'.hdarc'), 'w') as fp:
    fp.write(parameters)

In [30]:
from hda import Client

In [37]:
c = Client(debug=True)

2022-09-20 21:29:31,627 DEBUG HDA {'url': 'https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker', 'token': None, 'user': 'jftoro', 'password': 'gA2GHYbnYepfZR@', 'quiet': False, 'verify': True, 'timeout': None, 'sleep_max': 120, 'retry_max': 500, 'progress': True}


In [ ]:
# Import necessary libraries for the project

# Data
import pandas as pd

# Spatial data
# import geopandas as gpd
import rasterio as rio

# Machine learning

# Data visualization and widgets
import contextily as ctx
import matplotlib.pyplot as plt

# Requests (API's)
import requests
import json

In [38]:
query = {
  "datasetId": "EO:ESA:DAT:SENTINEL-3:OL_2_LFR___",
  "boundingBoxValues": [
    {
      "name": "bbox",
      "bbox": [
        -7.499583333333334,
        42.3020216796485,
        11.999333333333333,
        54.75429195536845
      ]
    }
  ],
  "dateRangeSelectValues": [
    {
      "name": "position",
      "start": "2021-01-01T00:00:00.000Z",
      "end": "2021-01-15T00:00:00.000Z"
    }
  ],
  "stringChoiceValues": [
    {
      "name": "productType",
      "value": "LFR"
    },
    {
      "name": "processingLevel",
      "value": "LEVEL2"
    }
  ]
}

# The following line runs the query
matches = c.search(query)

# The following line prints the products returned by the query
print(matches)

#The download starts. All the products found in the query are downloaded consecutively
matches.download()

2022-09-20 21:29:51,336 DEBUG ===> GET https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/gettoken
2022-09-20 21:29:51,350 DEBUG Starting new HTTPS connection (1): wekeo-broker.apps.mercator.dpi.wekeo.eu:443
2022-09-20 21:29:52,477 DEBUG https://wekeo-broker.apps.mercator.dpi.wekeo.eu:443 "GET /databroker/gettoken HTTP/1.1" 200 61
2022-09-20 21:29:52,484 DEBUG <=== {"access_token": "1c4b07cc-0d06-3423-aba9-73c3cc723803"}
2022-09-20 21:29:52,487 DEBUG Token is 1c4b07cc-0d06-3423-aba9-73c3cc723803
2022-09-20 21:29:52,490 DEBUG ===> POST https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/datarequest
2022-09-20 21:29:52,492 DEBUG ===> POST {"datasetId": "EO:ESA:DAT:SENTINEL-3:OL_2_LFR___", "boundingBoxValues": [{"na...
2022-09-20 21:29:52,808 DEBUG https://wekeo-broker.apps.mercator.dpi.wekeo.eu:443 "POST /databroker/datarequest HTTP/1.1" 200 93
2022-09-20 21:29:52,812 DEBUG <=== {"jobId": "oaECeYxL0rqtEPp9MDpV3sWj6dc", "status": "started", "results": [], ...
2022-09-20 21:

SearchResults[items=90,volume=6.8G,jobId=oaECeYxL0rqtEPp9MDpV3sWj6dc]


2022-09-20 21:30:02,900 DEBUG https://wekeo-broker.apps.mercator.dpi.wekeo.eu:443 "POST /databroker/dataorder HTTP/1.1" 200 80
2022-09-20 21:30:02,905 DEBUG <=== {"orderId": "7uBNAayu-K8VHZqwwv9WOMFyVtc", "status": "running", "message": null}
2022-09-20 21:30:02,909 DEBUG Sleeping 1 seconds
2022-09-20 21:30:03,916 DEBUG ===> GET https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/dataorder/status/7uBNAayu-K8VHZqwwv9WOMFyVtc
2022-09-20 21:30:04,338 DEBUG https://wekeo-broker.apps.mercator.dpi.wekeo.eu:443 "GET /databroker/dataorder/status/7uBNAayu-K8VHZqwwv9WOMFyVtc HTTP/1.1" 200 353
2022-09-20 21:30:04,343 DEBUG <=== {"status": "completed", "message": "Done!", "downloadUri": "3511eead-3261-5f7...
2022-09-20 21:30:04,346 INFO Downloading https://wekeo-broker.apps.mercator.dpi.wekeo.eu/databroker/dataorder/download/7uBNAayu-K8VHZqwwv9WOMFyVtc to S3A_OL_2_LFR____20210114T112412_20210114T112712_20210115T151755_0179_067_194_2160_LN1_O_NT_002.SEN3.zip (80.3M)
2022-09-20 21:30:05,026 D

KeyboardInterrupt: 

In [1]:
import snappy